In [ ]:
from stellargraph.mapper import (
    CorruptedGenerator,
    FullBatchNodeGenerator,
    GraphSAGENodeGenerator,
    HinSAGENodeGenerator,
    ClusterNodeGenerator,
)
from stellargraph import StellarGraph
from stellargraph.layer import GCN, DeepGraphInfomax, GraphSAGE, GAT, APPNP, HinSAGE

from stellargraph import datasets
from stellargraph.utils import plot_history

import pandas as pd
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
# from IPython.display import display, HTML

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras import Model

import numpy as np
import os
import csv
import sys

# PUT CVE CLASS
# cve = 'SAFE_23'
# edges_directory_path = 'CFGs/'+ cve +'_CFGs/edges_' + cve
edges_directory_path = sys.argv[1]

cve = 121


# PUT CVE CLASS
with open('/kaggle/input/features_matrices_' + cve + '.npy', 'rb') as f:
    features_matrices_list = np.load(f,  allow_pickle=True)

with open('/kaggle/input/nodes_targets_' + cve + '.npy', 'rb') as f:
    nodes_targets_list = np.load(f,  allow_pickle=True)



print('number of matrices', len(features_matrices_list))

graphs_representaions_path = '/graphs_representations_'+ cve +'.csv'

features_number = features_matrices_list[0].shape[1] #not used bec the embeddings size turns out to be 128

features_names = []
for i in range(128):
    features_names.append('X'+str(i))
features_names.append('y')

#if not os.path.exists(out_path):
#    os.mkdir(out_path)

# Using a with open() statement will automatically close a file once the block has completed
#if not os.path.exists(out_path + graphs_representaions_path):
#    with open(out_path + graphs_representaions_path, 'a', newline='') as file:
#            writer = csv.writer(file)
#            writer.writerow(features_names)


i = 0
graphs = []
for filename in os.listdir(edges_directory_path):
    f = os.path.join(edges_directory_path, filename)

    if os.path.isfile(f) and f.endswith('.csv'):
        # calling clear_session() when creating models in a loop to reset all state generated by Keras.
        tf.keras.backend.clear_session()

        # print(f)

        features_matrix = features_matrices_list[i]
        nodes_targets = nodes_targets_list[i]
        
        # Read in edges
        # if os.path.getsize(f) > 1000: 
        if os.stat(f).st_size > 0:
            edges = pd.read_csv(f)
            edges.columns = ['source', 'target'] # renaming for StellarGraph compatibility
            col=np.array(edges['target'], np.int16)
            edges['target']=col
            # print(edges)
            # print(edges.shape)

            node_features = pd.DataFrame(features_matrix)
            # print(node_features)
            # print(node_features.shape)

            G = StellarGraph(node_features, edges)
            graphs.append(G)

In [ ]:



import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import numpy as np
import stellargraph as sg

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Load the graph data and create a PyTorch Geometric DataLoader
def load_data(graphs):
    dataset = []
    for graph in graphs:
        node_features = graph.node_features()
        edge_index = torch.tensor(graph.to_adjacency_matrix().todense().nonzero(), dtype=torch.long)
        y = torch.tensor(graph.node_type, dtype=torch.long)
        x = torch.tensor(node_features.values, dtype=torch.float)
        data = Data(x=x, edge_index=edge_index, y=y)
        dataset.append(data)
    loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)
    return loader

# Train the GCN model on the graph data
def train_gcn(graphs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GCN(in_channels=10, out_channels=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loader = load_data(graphs)
    for epoch in range(10):
        for batch in loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = torch.nn.functional.cross_entropy(out, batch.y)
            loss.backward()
            optimizer.step()



In [ ]:

train_gcn(graphs)